In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd 
import seaborn as sns
import scipy

import warnings
warnings.filterwarnings("ignore")

# Genel Bakış

Bu kısımda veri seti üzerinde genel bir fikir kazanmak adına inceleme yapıldı:
- Veri setinin ilk 10 satırı yazdırıldı.
- Veri setinin kolonlarının isimleri yazdırıldı.
- Kolonların veri tipleri ve içerisindeki dolu veri sayısı yazdırıldı.

In [ ]:
# verinin okunması 
data = pd.read_csv("train.csv")
df = data.copy()

In [ ]:
#verinin ilk 10 satırını gösterir
df.head(10)

In [ ]:
df.info()

#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Kolonları inceleme aşamasında içerisinde nümerik değerler bulunan bazı değişkenlerin aslında kategorik tipte değişken olduğu fark edildi ve tip dönüşümü gerçekleştirildi.

In [ ]:
# sayısal veri içerdikleri için numerik gözüken değişkenlerin tipini kategorik tipine (object) çevrilmesi : 

df.loc[:,['MSSubClass', 'OverallQual', 'OverallCond', 'MoSold', 'YrSold']] = df.loc[:,['MSSubClass', 'OverallQual', 'OverallCond', 'MoSold', 'YrSold']].astype('object')

# Imputation

Bu aşamada genel olarak veri seti üzerindeki eksik ve hatalı verilerin temizlenmesi ve düzeltilmesi üzerine çalışıldı.

- Öncelikle excel tablosunun "**expectation**" kolonundaki nümerik verilerin değerlendirilmesini yapmak için korelasyon değerlerine bakıldı.

In [ ]:
# numerik değişkenler arasındaki korelasyonlara hızlı bir bakış : 

corr = df.corr()
f, ax = plt.subplots(figsize=(15, 12))
sns.heatmap(corr,linewidths=.5, vmin=0, vmax=1, square=True)

In [ ]:
# saleprice korelasyon matrisi:

k = 9 #number of variables for heatmap
corrmat = df.corr()
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(data[cols].values.T)
sns.set(font_scale=1.25)
plt.figure(figsize=(9,9))
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

### Null Verilerin Tespit Edilmesi ve Temizlenmesi

Öncelikle null veriler tespit edilerek toplam sayıları ve bu sayılara oranla yüzdeleri yazdırıldı.

In [ ]:
# yüzdesel olarak eksik verilerin kontrolü: 

total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

Bu bilgiler ışığında:
- PoolQC, MiscFeature, Alley, Fence ve FireplaceQu adlı kolonların null değerleri çok yüksek çıkmıştır.
- Örnek olarak PoolQC adlı kolon incelendiğinde toplamda 1460 değerin sadece 7 tanesi doludur. Geri kalan 1453 değer null veridir.

In [ ]:
df["PoolQC"].value_counts()

- Sonrasında yüzdesel olarak fazlaca eksik değere sahip olduğu için ve SalePrice kolonunu çok fazla etkilemeyeceği düşünüldüğü için veri setinden çıkarılmasına karar verilmiştir.
- Id kolonu da veri setinde bir anlam ifade etmediği için atıldı.

In [ ]:
# veri setinden çıkarılacak değişkenlerin çıkarılması: 

df=df.drop(['PoolQC','MiscFeature','Alley','Fence','FireplaceQu'], axis=1)

Belirtilen kolonlar silindikten sonra kalan null veriler tekrar yazdırıldı.

In [ ]:
# incelememiz gereken diğer değişkenleri görmek için tekrar eksik veri tablosunun gösterimi:

total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(15)

### Null verilerin doldurulması

Belirlediğimiz yüzdelik dilimin altında kalan kolonların null değerleri anlamlı verilerle dolduruldu.

LotFrontage kolonu evin sokağa bağlanma uzaklığını ifade etmektedir. Bu kolon üzerinde yapılan işlemler sonucu null değerlerin medyanla doldurulmasına karar verildi.

In [ ]:
# lot frontage

df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].median())

# Grouping Operations

Bu kısımda kolonları birleştirme işlemleri yapıldı.

Banyo sayılarıyla ilgili kolonlarda birleştirme işlemi yapıldı:
- FullBath; tam-banyo sayısını, HalfBath; yarım-banyo sayısını ifade etmektedir.
- BsmtFullBath; bodrumdaki tam-banyo sayısını, ve BsmtHalfBath; bodrumdaki yarım-banyo sayısını ifade etmektedir.
- TotalBath hesaplanırken FullBath için 1 katsayısı, HalfBath için 0.5 katsayısı kullanılarak hesaplanmıştır. Bu kolon toplam banyo sayısını ifade etmektedir.

In [ ]:
# TotalBath kolonunun hesaplanması ve ilgili kolonların çıkarılması 

df['TotalBath'] = df['FullBath'] + (0.5*df['HalfBath']) + df['BsmtFullBath'] + (0.5*df['BsmtHalfBath'])
df = df.drop(['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath'], axis = 1)

Veranda tipleri tek başlarına korelasyonları çok düşük olduğu için ve 0 sayısı çok fazla olduğu için TotalPorch adlı bir kolonda toplandı ve geri kalan kolonlar silindi.

In [ ]:
## TotalPorch kolonunun oluşturulması ve diğer ilgili kolonların silinmesi.

df["TotalPorch"] = df['OpenPorchSF'] + df['EnclosedPorch'] + df['3SsnPorch'] + df['ScreenPorch']
df = df.drop(['OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch'], axis = 1)

- Condition kolonları evin demiryollarına, park ve bahçeleri vs. yakınlığını göstermektedir. 
- Condition1 ve Condition2 kolonları çok sayıda aynı özelliği taşımaktadır.

In [ ]:
#Condition kolonlarının düzenlenmesi:

df[["Condition1","Condition2", "SalePrice"]]

- Condition2'de sadece 15 veri farklılık göstermektedir. Bu yüzden Condition2 kolonu atılarak sadece Conditon1 kolonu tutulacaktır.

In [ ]:
# condition1

df["Condition1"].value_counts()

In [ ]:
# condition2

df["Condition2"].value_counts()

In [ ]:
# condition2 değişkeni çıkarıldı, condition1 değişkeninin adı "condition" olarak değiştirildi:

df.drop("Condition2", axis=1, inplace=True)
df.rename(columns={"Condition1": "Condition"}, inplace=True)


* GarageArea ve GarageCars değişkenleri yüksek korele olduğu için ikisinden birini seçtik.
* GarageFinish değişkeninin bir etkisi olmayacağını düşünerek veri setinden çıkardık. 
* GarageYrBlt ile YearBuilt değişkeni yüksek korele olduğu için GarageYrBlt değişkenini veri setinden çıkardık.

* GarageType,GarageQual ve GarageCond değişkenleri NaN değerleri "NA"(No Garage) değeri ile dolduruldu

In [ ]:
# garageX
df = df.drop(['GarageArea','GarageYrBlt','GarageFinish'], axis = 1)

df['GarageType'].fillna('NA', inplace=True)
df['GarageQual'].fillna('NA', inplace=True)
df['GarageCond'].fillna('NA', inplace=True)

- BsmtQual, BsmtExposure ve BsmtCond kolonları diğer basement kolonlarını kapsadığından seçilmiş kolonlar kaldırıldı.
- Bu üç kapsayıcı kolonun NaN değerleri "NA"(No basement) değeri ile dolduruldu

In [ ]:
# basementX
df = df.drop(['BsmtFinSF2','BsmtUnfSF','BsmtFinSF2','BsmtFinSF1','BsmtFinType2','BsmtFinType1'], axis = 1)

df['BsmtExposure'].fillna('NA', inplace=True)
df['BsmtCond'].fillna('NA', inplace=True)
df['BsmtQual'].fillna('NA', inplace=True)

* MasVnrArea değişkeninde "0" olan noktaların MasVnrType değişkeninde karşılık olarak "NA" olarak doldurulmuştur.
* MasVnrType değişkeninde None olup MasVnrArea'da karşılığı 0 olan değişkenler tespit edilmiştir. Bu değerlerin outlier olarak düşünülüp outlier temizleme aşamasında çıkartılması gerekebilir.

In [ ]:
# masvnr
 
df.loc[df["MasVnrArea"]==0.0, "MasVnrType"] = "NA"

Electrical kolonundaki tek null değer çıkartıldı.

In [ ]:
# electrical :

df = df.drop(df.loc[df['Electrical'].isnull()].index)

1stFlrSF ve 2ndFlrSF değişkenlerinin toplamı GrLivArea değişkenini verdiği için bu iki değişken veri setinden çıkarıldı

In [ ]:
#1. kat alanı ve 2. kat alanı GrLivArea'yı verdiği için iki kolon da silindi.
df.drop(["1stFlrSF","2ndFlrSF"], axis=1, inplace=True)

#### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Alan bilgisi içeren kolonlardan 0 sayısı yüksek olanlar belirtildi.
- Bunun belirtilmesinin sebebi alan değeri 0 olduğunda o evin bu özelliğe sahip olmadığı bilgisine ulaşılırken, NaN olduğunda evin bu özelliğe sahip olup olmadığı bilinmemektedir.

In [ ]:
zeroSpaces = df[["LowQualFinSF", "PoolArea", "WoodDeckSF", "MiscVal", "MasVnrArea", "SalePrice"]]
zeroSpaces

Toplam 10 tane bol sıfırlı alan kolonu var, sadece bunların yüzdelerine bakıcak olursak:

In [ ]:
# kolonlardaki 0'ların yüzdesel olarak gösterimi: 

total = (df == 0).sum(axis=0).sort_values(ascending=False)
percent = ((df == 0).sum(axis=0)/df.count()).sort_values(ascending=False)
zeros = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
zeros.head(8)

Buradaki 0 değerleri yüksek kolonlar çıkartıldı.

In [ ]:
df = df.drop(['PoolArea','LowQualFinSF','MiscVal'], axis = 1)

In [ ]:
# veri setinin son hali : 

df.head()

Verileri temizledikten sonraki korelasyonları:

In [ ]:
# numerik değişkenler arasındaki korelasyonlara hızlı bir bakış : 

corr = df.corr()
f, ax = plt.subplots(figsize=(15, 12))
sns.heatmap(corr,linewidths=.5, vmin=0, vmax=1, square=True, annot=True)

# Detect Outliers

Öncelikle temizleme işlemlerinden sonra nümerik verilerin korelasyonlarına bakıldı.

In [ ]:
corr['SalePrice'].sort_values(ascending=False)

Korelasyon değeri rastgele 6 kolon seçildi ve görselleştirmeden faydalanarak outlier veriler incelendi.

In [ ]:
sns.set()
cols = ['SalePrice', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'TotalBath', 'YearBuilt','YearRemodAdd']
sns.pairplot(df[cols], size = 2.5)
plt.show();

Buradan sonra bu kolonların 4 tanesinin outlierları temizlenecektir. Outlier temizleme işlemi için IQR seçilmiştir.

Seçilen kolonların IQR yöntemiyle outlier tespiti:

In [ ]:
selected_columns = df.select_dtypes(include=np.number).columns.tolist()[1:]
numeric_df = df[selected_columns]

In [ ]:
numeric_df_statsinfo = numeric_df.describe()

def showstats(dataFrame_statsinfo,column_name):

    mean   = dataFrame_statsinfo[column_name][1]
    median = dataFrame_statsinfo[column_name][5]
    median_mean_ratio = median/mean
    Q1     = dataFrame_statsinfo[column_name][4]
    Q3     = dataFrame_statsinfo[column_name][6]
    IQR    = Q3-Q1
    upper_band = Q3 + 1.5*IQR
    lower_band = Q1 - 1.5 *IQR

    print('\n' + str(column_name)+'\n' +
          "\n" "ortalama : " + str(mean) +
          '\n' "medyan    : " + str(median) +
          '\n' "Q1 değeri    : " + str(Q1) +
          '\n' "Q3 değeri    : " + str(Q3) + 
          '\n' "IQR değeri    : " + str(IQR) + 
          '\n' "alt ve üst değer aralığı  : " + str((lower_band,upper_band))
         )

    return upper_band , lower_band 


for column_name in selected_columns :
    upper_band , lower_band = showstats(numeric_df_statsinfo , column_name)
    outliers = numeric_df[(numeric_df[column_name] < lower_band) | (numeric_df[column_name] > upper_band)][column_name].values
    print('\n' + str(column_name) + " değişkeninin " + str(len(outliers)) + " outlier noktası vardır  ")
    print('\n\n****************************************************************************\n')

Kategorik değişkenler için Box-Plot

In [ ]:
num_col=df._get_numeric_data().columns.tolist()[1:]
cat_col=set(df.columns)-set(num_col)
cat_col.remove("Id")

for cols in cat_col:
    f, ax = plt.subplots(figsize=(8, 6))
    fig = sns.boxplot(x=cols, y="SalePrice", data=df)
    fig.axis(ymin=0, ymax=800000);
    plt.show()

Nümerik değişkenler için Histogram-Plot

In [ ]:
for cols_ in num_col:
    f, ax = plt.subplots(figsize=(8, 8))
    fig =sns.distplot(df[cols_])